In [ ]:
import cobra
import pandas as pd
import riptide
import os

def parameterize_model(model, transcriptomic):
    # Parameterize model using Riptide
    tr = riptide.read_transcription_file(transcriptomic, header=False)
    print(tr)
    output = riptide.maxfit(model, tr)
    return output

def save_dict_as_tsv(data_dict, output_path):
    # Handling both scalar and list-like dictionary values
    if all(isinstance(value, (int, float, str)) for value in data_dict.values()):
        df = pd.DataFrame(list(data_dict.values()), index=data_dict.keys(), columns=['Value'])
    else:
        df = pd.DataFrame(data_dict)
    df.to_csv(output_path, sep='\t', index=False)
    print(f"Data saved at {output_path}")

# Load model and configure medium
model = cobra.io.read_sbml_model('/Users/yahyafarooqi/Documents/Code/AntiGEM/FluxGraph/Models/Standard/subtilis_ iYO844.xml')
medium = model.medium
medium['EX_glc__D_e'] = 10
medium['EX_trp__L_e'] = 1
medium['EX_glu__L_e'] = 1
medium['EX_cit_e'] = 1
model.medium = medium

# Paths and directories setup
path = '/Users/yahyafarooqi/Documents/Code/AntiGEM/FluxGraph/Data/intermediate/transposed_dataframes_filled'
output_directory_1 = '/Users/yahyafarooqi/Documents/Code/AntiGEM/FluxGraph/Data/intermediate/carbon_fluxes'
output_directory_2 = '/Users/yahyafarooqi/Documents/Code/AntiGEM/FluxGraph/Data/intermediate/carbon_fit'
model_save_directory = '/Users/yahyafarooqi/Documents/Code/AntiGEM/FluxGraph/Data/clean/c_models'

# Ensure output directories exist
os.makedirs(output_directory_1, exist_ok=True)
os.makedirs(output_directory_2, exist_ok=True)
os.makedirs(model_save_directory, exist_ok=True)

# Process each .tsv file in the directory
for filename in os.listdir(path):
    if filename.endswith('.tsv'):

        # Load the transcriptomic data
        file_path = os.path.join(path, filename)
        # Assuming parameterize_model returns an object with attributes 'model' and 'concordance'
        obj = parameterize_model(model, file_path)
        if obj:
            c_model = obj.model

            # Save the constrained model as an XML file
            #remove the .tsv extension
            filename = filename.replace('.tsv', '')
            
            model_save_path = os.path.join(model_save_directory, f'{filename}_constrained.xml')
            cobra.io.write_sbml_model(c_model, model_save_path)
            print(f"Constrained model saved as XML for {filename}")

            # Perform flux balance analysis and save results

        try:
            solution = cobra.flux_analysis.pfba(c_model)
            flux_output_file_path = os.path.join(output_directory_1, f'{filename}_fluxes.tsv')
            solution.fluxes.to_csv(flux_output_file_path, sep='\t')
            print(f"Flux balance analysis successful for {filename}. Results saved to {flux_output_file_path}")
        except cobra.exceptions.Infeasible:
            print(f"Warning: FBA is infeasible for model {filename}. Skipping this model.")

            # Save concordance dictionary
            fit_output_file_path = os.path.join(output_directory_2, f'{filename}_fit.tsv')
            save_dict_as_tsv(obj.concordance, fit_output_file_path)



{'BSU26790': [329.8514093905791, 329.8514093905791], 'BSU27090': [224.92908718963935, 200.11041105217916], 'BSU28720': [190.0608998790825, 157.79789386861364], 'BSU14480': [1841.1522961191363, 2719.6725594340937], 'BSU28810': [201.55457582532932, 203.98916699784942], 'BSU00370': [96.04482038099758, 228.11739105076813], 'BSU29200': [192.02775876574813, 119.71674479735464], 'BSU24350': [155.09420592895617, 147.20949797244268], 'BSU24340': [171.9095431375909, 189.4871468328349], 'BSU37170': [344.2581890729287, 176.35052284333005], 'BSU29470': [257.42148589978063, 216.83012968744083], 'BSU08060': [294.6753814186833, 294.6753814186833], 'BSU08070': [250.815988438717, 207.8596340258369], 'BSU08080': [369.9192891766633, 369.9192891766633], 'BSU08090': [241.00720504173174, 173.90515223288162], 'BSU08100': [358.85241205656496, 218.31020771232312], 'BSU15920': [144.84103815673856, 183.30077804358163], 'BSU29680': [210.0368214284135, 244.26344720208567], 'BSU29690': [178.6581075019464, 221.754591

AttributeError: 'str' object has no attribute 'values'